# Project management

In [33]:
from os import path, getenv
from mlrun import new_project

project_name = '-'.join(filter(None, ['properties-management']))
project_path = path.abspath('conf')
project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/igztraining/mlrun/Day-4/conf
Project name: properties-management


In [34]:
from mlrun import run_local, NewTask, mlconf, import_function, mount_v3io

# Target location for storing pipeline artifacts
artifact_path = path.abspath('jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/igztraining/mlrun/Day-4/jobs
MLRun DB path: http://mlrun-api:8080


# Deploy code as an MLRun function

In [ ]:
from mlrun import code_to_function, mlconf,mount_v3io

In [ ]:
fn = code_to_function(project='Iguazio-Training',name='properities-handler', kind='job',filename='modules/mlrun_functions.ipynb')

In [ ]:
fn.spec.image = "mlrun/mlrun"

In [ ]:
fn.apply(mount_v3io())

# Add function to project

In [ ]:
project.set_function(fn,name='properties-handler')

In [ ]:
project.functions

### Run interactive

In [ ]:
params = {"source_file" : "/User/igztraining/mlrun/Day-4/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/Day-4/data/condos.csv" }

In [ ]:
save_properties = project.func('properties-handler').run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training')

### Run step 2 with Step 1's output

In [ ]:
params['city']="SACRAMENTO"
params['target_dataset']="sacramento_condos"

In [ ]:
sp_run = project.func('properties-handler').run(name='run_withcode',artifact_path="/User/artifacts",handler='filter_properties',
                inputs={"source_data":save_properties.outputs["condo_properties"]},
                params=params,project='Iguazio-Training')

In [ ]:
project.save()

# Run function as part of a pipeline

In [ ]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [31]:
@dsl.pipeline(
    name = 'Property Management',
    description = 'Condo reviews'
)

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())


def job_pipeline(
    source_file :str = ""
) -> None:
    
    save_properties = project.func('properties-handler').as_step(handler='save_property',
                            params={"source_file" : source_file,
                                    "target_file" : "/User/igztraining/mlrun/data/condos.csv" },
                            outputs=['condo_properties'])
    
    
    filter_properties = project.func('properties-handler').as_step(handler='filter_properties',
                                   params={"city" : "SACRAMENTO",
                                           "target_dataset" : "sacramento_condos" },
                                   inputs={"source_data":save_properties.outputs["condo_properties"]})

In [32]:
kfp.compiler.Compiler().compile(job_pipeline, 'jobpipe.yaml')
artifact_path = 'v3io:///users/admin/kfp/{{workflow.uid}}/'
arguments = {"source_file" : "/User/igztraining/mlrun/Day-4/data/demo.csv"}
run_id = run_pipeline(job_pipeline, arguments, experiment='property_management', artifact_path=artifact_path)

> 2020-09-01 15:04:08,442 [info] Pipeline run id=1c53bfba-05e0-4b86-921a-f456a27b8f6c, check UI or DB for progress
